In [1]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
from dataclasses import dataclass
import requests
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
SLEEP = 2
TOPICS = 'turizm'
sort ='weight'
date_from = '2000-01-01'
date_to = '2024-01-04'
offset = 1


In [3]:
param_dict = {'rubric'     : TOPICS,
              'sort'      : sort,
              'dateFrom'  : date_from,
              'dateTo'    : date_to,
             'offset': offset}

print("param_dict:", param_dict)

param_dict: {'rubric': 'turizm', 'sort': 'weight', 'dateFrom': '2000-01-01', 'dateTo': '2024-01-04', 'offset': 1}


In [4]:
class fontanka_parser:

    def __init__(self):
        ...
    
    
    def get_url(self, param_dict: dict) -> str:
        url = 'https://www.fontanka.ru/cgi-bin/search.scgi?'\
        + 'rubric={}&'.format(param_dict['rubric'])\
        + 'sortt={}&'.format(param_dict['sort'])\
        + 'fdate={}&'.format(param_dict['dateFrom'])\
        + 'tdate={}&'.format(param_dict['dateTo'])\
        + 'offset={}&'.format(param_dict['offset'])\
        
        
        return url
    
    def append_info(self,param_dict,offset=1):
        #offset - страница парсинга, сколько укажем столько и спарсится
        global info
        # получения ссылки
        for off in tqdm(range(1,offset+1)):
            # присваиваем смещение 
            param_dict['offset'] = off
            url = self.get_url(param_dict)
        
            #получение эл-ов класса
            response = requests.get(url)
            print(response.status_code)
            print(url)
            tree = BeautifulSoup(response.content, 'html.parser')
            ex=tree.find_all('div', {'class': 'IXagh'})
            url_f_cont = 'https://www.fontanka.ru/'
            
            for i in tqdm(range(len(ex))):
                
                try:
                    #time.sleep(SLEEP)
                    #получения объекта i
                    url_post=ex[i].a.get('href')
                    if re.search(r'https://www.fontanka.ru',url_post):
                        post = requests.get(url_post)
                    else:
                        post = requests.get(url_f_cont+url_post)
                    
                    # запрос на поиск инфы текста
                    post_soup = BeautifulSoup(post.content, 'html.parser')
                    post_ex=post_soup.find('div', {'class': 'KThj'})
            
                    #получаем текст поста
                    text = post_ex.text
                    
                    #получаем тематику поста
                    title = ex[0].find('a', {'class': "IXil IXct"}).get('title')
                    
                    row = {
                        'title': title,
                        'text': text
                        }
                    #доавление в список
                    info.append(row)
            
                except Exception as exept:
                    print(i,exept)
        
            # сообщение об успешном сохранении данных страниц
            print(f'Checkpoint page {off} saved!')

In [5]:
# задаем топики, по которым будем парсить
topics = ['society']

In [6]:
info=[]
for i in topics:
    print(i)
    param_dict['rubric'] =i
    obj = fontanka_parser()
    obj.append_info(param_dict, offset=1)

society


  0%|          | 0/1 [00:00<?, ?it/s]

200
https://www.fontanka.ru/cgi-bin/search.scgi?rubric=society&sortt=weight&fdate=2000-01-01&tdate=2024-01-04&offset=1&



 37%|███▋      | 11/30 [00:21<00:36,  1.92s/it]

10 'NoneType' object has no attribute 'text'


...

12 'NoneType' object has no attribute 'text'



 70%|███████   | 21/30 [00:39<00:14,  1.66s/it]

20 'NoneType' object has no attribute 'text'



100%|██████████| 1/1 [00:58<00:00, 58.47s/it]

Checkpoint page 1 saved!


In [7]:
df = pd.DataFrame(info)
#df3 = pd.DataFrame(info3)
df.tail()

,title,text
22,Общество,Фото: стоп-кадр / Киноконцерн «Мосфильм» / You...
23,Общество,Уголовное дело об уклонении от уплаты налогов ...
24,Общество,Право на отсрочку от частичной мобилизации пол...
25,Общество,Актёр Михаил Боярский уже находится дома после...
26,Общество,Сборы российских кинотеатров по итогам 2023 го...
